In [ ]:
!git clone "https://github.com/timothybrooks/instruct-pix2pix/"

In [ ]:
cd /kaggle/working/instruct-pix2pix

In [ ]:
print("\nInstalling Dependencies ...")

In [ ]:
!conda env create -f /kaggle/working/instruct-pix2pix/environment.yaml > /dev/null 2>&1

In [ ]:
print("\nDependencies Installed")

In [ ]:
mkdir /root/.kaggle/

In [ ]:
cp -r /kaggle/input/api-connections/kaggle2.json /root/.kaggle/kaggle.json

In [ ]:
cp -r /kaggle/input/api-connections/service_account.json /kaggle/working/instruct-pix2pix

In [ ]:
cp -r /kaggle/input/pix2pix-checkpoints/checkpoints /kaggle/working/instruct-pix2pix

In [ ]:
import os
import subprocess
import shutil
import json
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload


DATASET_PATH = "/kaggle/working/dataset"

class GoogleDrive_API:
    def __init__(self):
        self.SCOPES = ["https://www.googleapis.com/auth/drive"]
        self.SERVICE_ACCOUNT_FILE = "service_account.json"
        self.PARENT_FOLDER_ID = "1r-MlnEpWHx3b1fxHDnHcZ2-Wh_Y89676"
        self.service = self.authenticate()

    def authenticate(self):
        credentials = service_account.Credentials.from_service_account_file(
            self.SERVICE_ACCOUNT_FILE, scopes=self.SCOPES
        )
        service = build("drive", "v3", credentials=credentials)

        return service

    def get_files(self):
        # List all files in the folder
        results = (
            self.service.files()
            .list(
                q=f"'{self.PARENT_FOLDER_ID}' in parents and trashed=false",
                fields="files(id, name)",
            )
            .execute()
        )
        return results.get("files", [])

    def upload_file(self, file_path):
        file_metadata = {
            "name": os.path.basename(file_path),
            "parents": [self.PARENT_FOLDER_ID],
        }
        media = MediaFileUpload(file_path)
        file = (
            self.service.files()
            .create(body=file_metadata, media_body=media, fields="id")
            .execute()
        )
        print("File uploaded. File ID:", file.get("id"))

    def download_file(self, file_name, file_path):
        results = (
            self.service.files()
            .list(
                q=f"'{self.PARENT_FOLDER_ID}' in parents and name='{file_name}' and trashed=false",
                fields="files(id)",
            )
            .execute()
        )
        items = results.get("files", [])

        if items:
            # Get the file ID
            file_id = items[0]["id"]

            request = self.service.files().get_media(fileId=file_id)

            with open(file_path, "wb") as file:
                downloader = MediaIoBaseDownload(file, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%.")
        else:
            print(rf"{file_name} is Not Found")

def execute_terminal_command(command: str):
    try:
        process = subprocess.Popen(
            command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE
        )
        output, error = process.communicate()
        output = str(output.decode("utf-8"))
        error = str(error.decode("utf-8"))
        print(rf"Command executed successfully: {command}")
        return output
    except Exception as e:
        print("Command Failed !")
        return None, str(e)
        
def read_file(file_path: str):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data[0]

def write_file(data: list, file_path: str, file_name: str = ""):
    # Writing JSON data
    with open(rf"{file_path}/{file_name}", "w") as file:
        for idx in range(len(data)):
            json_string = json.dumps(data[idx]) + ("\n" if idx < len(data) - 1 else "")
            file.write(json_string)
            
def read_image(image_path: str):
    try:
        image = Image.open(image_path)
        return image
    except IOError:
        print("Unable to load image")
        return None
    
def generate_image(input_image, edit_instruction, output_path, steps, seed, cfgtext, cfgimage, resolution):
    inputs = fr'--input {input_image} --output {output_path} --edit "{edit_instruction}" --steps {steps} --seed {seed} --cfg-text {cfgtext} --cfg-image {cfgimage} --resolution {resolution}'
    command = fr"/opt/conda/envs/ip2p/bin/python3 edit_cli.py  {inputs}"
    return execute_terminal_command(command)


def main():
    if not os.path.exists(DATASET_PATH):
        execute_terminal_command(rf"mkdir {DATASET_PATH}")
    
    # download dataset
    GoogleDrive_connection = GoogleDrive_API()
    GoogleDrive_connection.download_file("data.json", rf"{DATASET_PATH}/data.json")
    
    # input
    data = read_file(rf"{DATASET_PATH}/data.json")
    # get data
    time = data["time"]
    print(fr"Start Time: {time}")
    edit_instruction = data["edit_instruction"]
    input_image_path = data["input_image_path"]
    output_image_path = data["output_image_path"]
    steps = data["steps"]
    seed = data["seed"]
    cfgtext = data["cfg-text"]
    cfgimage = data["cfg-image"]
    resolution = data["resolution"]

    GoogleDrive_connection.download_file(input_image_path, rf"{DATASET_PATH}/{input_image_path}")

    input_image = read_image(rf"{DATASET_PATH}/{input_image_path}")
    # run model
    ######################################################
    generate_image(rf"{DATASET_PATH}/{input_image_path}", edit_instruction, rf"{DATASET_PATH}/{output_image_path}", steps, seed, cfgtext, cfgimage, resolution)
    ######################################################
    
    # update dataset
    GoogleDrive_connection.upload_file(rf"{DATASET_PATH}/{output_image_path}")
    
    # Load the images
    img1 = mpimg.imread(rf"{DATASET_PATH}/{input_image_path}")
    img2 = mpimg.imread(rf"{DATASET_PATH}/{output_image_path}")

    # Display the images side by side
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(img1)
    plt.axis('off')
    plt.title('Input Image')

    plt.subplot(1, 2, 2)
    plt.imshow(img2)
    plt.axis('off')
    plt.title(rf"Edited Image: {edit_instruction}")
        
    plt.show()
        
if __name__ == "__main__":
    main()